# Database for the Recipies 

Import all the Important Modules

In [1]:
import json
import os
import sqlite3

In [2]:
#connection to DB
con = sqlite3.connect("RecipeDB.db")

#create cursor
cursor = con.cursor()

![alt text](Database.png "Datenbank")


Creating the Database


In [3]:


#Recipe Table
sql = """
CREATE TABLE Recipe(
recID TEXT PRIMARY KEY NOT NULL,
recName TEXT,
recHeadline TEXT,
recDisc BLOB,
recDifficulty INT,
recPrepTime TEXT, 
recTotalTime TEXT,
recServingSize INT,
recLink TEXT,
recRating REAL,
recRatingCount INT,
recSteps INT
);
"""

cursor.execute(sql)

#IngredientFamily Table

sql = """
CREATE TABLE IngFamily(
familyID TEXT PRIMARY KEY NOT NULL,
familyName TEXT
);

"""

cursor.execute(sql)

#Ingredients TAble
sql = """
CREATE TABLE Ingredients(
ingID TEXT PRIMARY KEY NOT NULL,
ingName TEXT NOT NULL,
familyID TEXT NOT NULL,
FOREIGN KEY(familyID) REFERENCES IngFamily(familyID)
);

"""

cursor.execute(sql)


#Ingredients in Recipe Table
sql = """
CREATE TABLE IngInRec(
ingID TEXT NOT NULL,
recID TEXT NOT NULL,
amount REAL,
unit TEXT,
PRIMARY KEY(ingID, recID, unit),
FOREIGN KEY (ingID) REFERENCES Ingredients(ingID),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);

"""
cursor.execute(sql)

#Nutartion Table plus NutrInRec
sql = """
CREATE TABLE Nutrations(
nType TEXT PRIMARY KEY NOT NULL,
nName TEXT NOT NULL
);
"""

cursor.execute(sql)

sql = """
CREATE TABLE NutrInRec(
nType TEXT NOT NULL,
recID TEXT NOT NULL,
amount INT,
unit TEXT,
PRIMARY KEY(nType, recID),
FOREIGN KEY (nType) REFERENCES Nutration(nType),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);
"""

cursor.execute(sql)


#Tags and TagsInRecipe

sql = """
CREATE TABLE Tags(
tagID TEXT PRIMARY KEY NOT NULL,
tagName TEXT,
tagType TEXT
);

"""

cursor.execute(sql)

sql = """
CREATE TABLE TagsInRec(
tagID TEXT NOT NULL,
recID TEXT NOT NULL,
PRIMARY KEY (tagID, recID),
FOREIGN KEY (tagID) REFERENCES Tags(tagID),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);

"""

cursor.execute(sql)

#Allergens and it´s conTable
sql = """
CREATE TABLE Allergens(
alleID TEXT PRIMARY KEY NOT NULL,
alleType TEXT,
alleName TEXT,
alleSlug TEXT,
alleDisc BLOP,
allTraces BOOLEAN NOT NULL CHECK(allTraces IN (0,1)),
triggersTracesOf BOOLEAN NOT NULL CHECK(triggersTracesOf IN (0,1))

);

"""
cursor.execute(sql)

sql = """
CREATE TABLE AlleInRec(
alleID TEXT NOT NULL,
recID TEXT NOT NULL,
PRIMARY KEY (alleID,recID),
FOREIGN KEY (alleID) REFERENCES Allergens(alleID),
FOREIGN KEY (recID) REFERENCES Recipe(recID)

);

"""
cursor.execute(sql)

#steps table
sql = """
CREATE TABLE StepsInRec(
recID TEXT NOT NULL,
stepIndex INT NOT NULL,
stepDisc BLOP,
PRIMARY KEY (recID, stepIndex),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);
"""
cursor.execute(sql)

Inserting the Date from the JSON files

In [3]:

test = cursor.execute(""" SELECT * FROM Recipe WHERE recID = '554b1a294dab71a6058b4570' """).fetchall()
print(len(test))

0


In [4]:
path = os.getcwd()+"/rezepte"
#sql to check if there is * already in DB
recipe_check = """ SELECT * FROM Recipe WHERE recID = ? ;"""
ingFamCheck = """SELECT * FROM IngFamily WHERE familyID = ? ;"""
ing_check = """ SELECT * FROM Ingredients WHERE ingID = ? ;"""
Nutr_check = """SELECT * FROM Nutrations WHERE nType = ? ;"""
allg_check = """ SELECT * FROM Allergens WHERE alleID = ? ; """
tag_check = """SELECT * FROM Tags WHERE tagID = ? ;"""

#insert sql
recipe_sql =""" INSERT INTO Recipe VALUES (?,?,?,?,?,?,?,?,?,?,?,?);""" 
ingFam_sql = """INSERT INTO IngFamily VALUES (?,?);"""
ing_sql = """INSERT INTO Ingredients VALUES (?,?,?);""" 
alle_sql = """INSERT INTO Allergens VALUES (?,?,?,?,?,?,?);""" 
nutr_sql = """INSERT INTO Nutrations VALUES (?,?);"""
tags_sql = """INSERT INTO Tags VALUES (?,?,?);""" 
steps_sql = """INSERT INTO StepsInRec VALUES (?,?,?);""" 
air_sql = """INSERT INTO AlleInRec VALUES (?,?);"""
iir_sql = """INSERT INTO IngInRec VALUES (?,?,?,?);"""
nir_sql = """INSERT INTO NutrInRec VALUES (?,?,?,?);"""
tir_sql = """INSERT INTO TagsInRec VALUES (?,?);""" 




without = 0
for f in os.listdir(path):
    with open(os.path.join(path,f), encoding = 'utf8') as file:
        data = json.load(file)
        
        #check if recipe has ingredients
        if not data['ingredients']:
            without += 1
            continue
            
        #check if recipeID is already in DB
        if len(cursor.execute(recipe_check,(data['id'],)).fetchall()):
            without -= -1
            continue
            
        #add recipe
        temp = (data['id'],data['name'],data['headline'],data['description'],data['difficulty'],data['prepTime'],data['totalTime'],data['servingSize'],data['cardLink'],data['averageRating'], data['ratingsCount'],len(data['steps']))
        cursor.execute(recipe_sql, temp)
        #add ingredients and all its tables
        
        for ing in data['ingredients']:
            if len(cursor.execute(ing_check,(ing['id'],)).fetchall()):
                continue
            if not len(cursor.execute(ingFamCheck,(ing['family']['id'],)).fetchall()):
                fam = ing['family']
                temp = (fam['id'],fam['name'])
                cursor.execute(ingFam_sql,temp)
            
            temp = (ing['id'],ing['name'],ing['family']['id'])
            cursor.execute(ing_sql,temp)
        
        #TODO make a check cuz the ingriedients per recipe arent unique bruuuuuuuuuuuuuuuuuuh
        y = data['yields'][0]
        div = int(y['yields'])
        for ing in y['ingredients']:
            print(ing)
            am = int(ing['amount']) if ing['amount'] else 0
            am /= div
            temp = (ing['id'],data['id'],am,ing['unit'])
            cursor.execute(iir_sql,temp)
        
        #nutr
        for n in data['nutrition']:
            if not len(cursor.execute(Nutr_check,(n['type'],)).fetchall()):
                cursor.execute(nutr_sql,(n['type'],n['name']))
            cursor.execute(nir_sql,(n['type'],data['id'],n['amount'],n['unit']))
        
        #Tags
        for t in data['tags']:
            if not len(cursor.execute(tag_check,(t['id'],)).fetchall()):
                cursor.execute(tags_sql,(t['id'],t['name'],t['type']))
            cursor.execute(tir_sql,(t['id'],data['id']))
        
        #Allergens
        for a in data['allergens']:
            if not len(cursor.execute(allg_check,(a['id'],)).fetchall()):
                cursor.execute(alle_sql,(a['id'],a['type'],a['name'],a['slug'],a['description'],a['tracesOf'],a['triggersTracesOf']))
            cursor.execute(air_sql,(a['id'],data['id']))
        
        
        #Steps
        for s in data['steps']:
            cursor.execute(steps_sql,(data['id'],s['index'],s['instructions']))
        
        con.commit()
            
cursor.close()
        
        
        



{'id': '554b7698fd2cb96c628b4568', 'amount': 32, 'unit': 'g'}
{'id': '55a7a8f8fd2cb9570f8b4567', 'amount': 23, 'unit': 'g'}
{'id': '556c4ed44dab710b3d8b4569', 'amount': 23, 'unit': 'g'}
{'id': '55535c2efd2cb9ae048b4567', 'amount': 23, 'unit': 'g'}
{'id': '556db2a04dab71104b8b4568', 'amount': 2, 'unit': 'Stück'}
{'id': '554b6070f8b25e49448b4568', 'amount': 300, 'unit': 'g'}
{'id': '554b7f33fd2cb96c628b456a', 'amount': 10, 'unit': 'g'}
{'id': '554b268ffd2cb983608b4568', 'amount': 0.125, 'unit': 'Stück'}
{'id': '554b1de2f8b25ef93e8b456b', 'amount': 1, 'unit': 'Stück'}
{'id': '565574b64dab71665f8b4567', 'amount': 1, 'unit': 'g'}
{'id': '556c7ea0f8b25e106c8b4567', 'amount': 20, 'unit': 'g'}
{'id': '556c09b7f8b25e74628b4567', 'amount': 1, 'unit': 'Stück'}
{'id': '554b34554dab71a6058b4576', 'amount': 0.5, 'unit': 'Stück'}
{'id': '556c5b19fd2cb97a4d8b4567', 'amount': 0.5, 'unit': 'Stück'}
{'id': '554b1e654dab71a6058b4571', 'amount': 5, 'unit': 'g'}
{'id': '56273fd94dab713a1d8b4567', 'amount': 

{'id': '554b34554dab71a6058b4576', 'amount': 1, 'unit': 'Einheit'}
{'id': '55534596f8b25ea1038b4569', 'amount': 10, 'unit': 'g'}
{'id': '554b1bf6f8b25eef3e8b4570', 'amount': 0.5, 'unit': 'Esslöffel'}
{'id': '55683ad84dab710f1c8b456f', 'amount': 400, 'unit': 'g'}
{'id': '55683b114dab71c91c8b4567', 'amount': 250, 'unit': 'g'}
{'id': '555dabaff8b25e246a8b4568', 'amount': 100, 'unit': 'g'}
{'id': '554b2288f8b25ef93e8b456c', 'amount': 1, 'unit': 'Esslöffel'}
{'id': '55683c414dab71a41c8b4568', 'amount': 1, 'unit': 'g'}
{'id': '554b1c284dab71a9058b456b', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1c56fd2cb99c598b4573', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1f6efd2cb99c598b4575', 'amount': 200, 'unit': 'g'}
{'id': '554b1c284dab71a9058b456b', 'amount': None, 'unit': 'Einheit'}
{'id': '554b7b9efd2cb9d0608b456f', 'amount': 250, 'unit': 'g'}
{'id': '556840d24dab71d81c8b4567', 'amount': 0.5, 'unit': 'Einheit'}
{'id': '554b1adef8b25ef93e8b456a', 'amount': 1, 'unit': 'Einheit'}
{'id': '5

{'id': '554b6070f8b25e49448b4568', 'amount': 2, 'unit': 'Einheit'}
{'id': '554b25fcfd2cb973608b4567', 'amount': 600, 'unit': 'g'}
{'id': '554b268ffd2cb983608b4568', 'amount': 0.125, 'unit': 'Einheit'}
{'id': '554b1de2f8b25ef93e8b456b', 'amount': 0.5, 'unit': 'Einheit'}
{'id': '554b1a5b4dab71c8058b456d', 'amount': 1, 'unit': 'Zweig'}
{'id': '554b2645fd2cb983608b4567', 'amount': 1, 'unit': 'g'}
{'id': '554b2aa14dab71aa058b4570', 'amount': 1, 'unit': 'Einheit'}
{'id': '554b1adef8b25ef93e8b456a', 'amount': 0.5, 'unit': 'Einheit'}
{'id': '554b34b64dab71aa058b4573', 'amount': 5, 'unit': 'g'}
{'id': '554b7480fd2cb911618b4568', 'amount': 75, 'unit': 'g'}
{'id': '554b22cb4dab71c8058b4570', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1c284dab71a9058b456b', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1c56fd2cb99c598b4573', 'amount': None, 'unit': 'Einheit'}
{'id': '554b7d614dab71160d8b456b', 'amount': None, 'unit': 'Einheit'}
{'id': '556881724dab71111f8b4568', 'amount': 2, 'unit': 'Einheit'

{'id': '556735f14dab71190e8b4567', 'amount': 4, 'unit': 'Einheit'}
{'id': '554b25fcfd2cb973608b4567', 'amount': 400, 'unit': 'g'}
{'id': '554b34b64dab71aa058b4573', 'amount': 10, 'unit': 'g'}
{'id': '5566ebf34dab71e6078b456b', 'amount': 1, 'unit': 'Einheit'}
{'id': '554b1de2f8b25ef93e8b456b', 'amount': 0.5, 'unit': 'Einheit'}
{'id': '556c5f24fd2cb97b4d8b4567', 'amount': 100, 'unit': 'g'}
{'id': '554b3300fd2cb973608b4569', 'amount': 100, 'unit': 'ml'}
{'id': '5566f045fd2cb952018b456b', 'amount': 1, 'unit': 'g'}
{'id': '556c5fa6fd2cb98b4d8b4567', 'amount': 50, 'unit': 'g'}
{'id': '554b1c284dab71a9058b456b', 'amount': None, 'unit': 'Einheit'}
{'id': '554b22cb4dab71c8058b4570', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1c56fd2cb99c598b4573', 'amount': None, 'unit': 'Einheit'}
{'id': '554b3354fd2cb9ba598b456e', 'amount': None, 'unit': 'Einheit'}
{'id': '555dc1a9fd2cb944758b4569', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1bf6f8b25eef3e8b4570', 'amount': None, 'unit': 'Einheit'}
{'

{'id': '554b7b9efd2cb9d0608b456f', 'amount': 250, 'unit': 'g'}
{'id': '55535721f8b25e8a038b4569', 'amount': 200, 'unit': 'g'}
{'id': '554b268ffd2cb983608b4568', 'amount': 0.125, 'unit': 'Einheit'}
{'id': '555da357fd2cb943758b4567', 'amount': 5, 'unit': 'g'}
{'id': '55683ad84dab710f1c8b456f', 'amount': 500, 'unit': 'g'}
{'id': '554b3300fd2cb973608b4569', 'amount': 100, 'unit': 'ml'}
{'id': '556c48704dab710b3d8b4568', 'amount': 100, 'unit': 'g'}
{'id': '556d8a7c4dab713d4a8b4567', 'amount': 1, 'unit': 'Einheit'}
{'id': '554b22cb4dab71c8058b4570', 'amount': None, 'unit': 'Einheit'}
{'id': '554b2288f8b25ef93e8b456c', 'amount': None, 'unit': 'Einheit'}
{'id': '555dc1a9fd2cb944758b4569', 'amount': None, 'unit': 'Einheit'}
{'id': '55535e24f8b25e78738b4569', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1c284dab71a9058b456b', 'amount': None, 'unit': 'Einheit'}
{'id': '554b1c56fd2cb99c598b4573', 'amount': None, 'unit': 'Einheit'}
{'id': '554b2a87f8b25ef53e8b4573', 'amount': 200, 'unit': 'g'}
{'

IntegrityError: UNIQUE constraint failed: IngInRec.ingID, IngInRec.recID, IngInRec.unit